In [ ]:
import requests
import json
from IPython.display import JSON
from keys import *
#estas lineas son para importar lo que se va a utilizar en el codigo


In [ ]:
#estas son para hacer la peticion a la API

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/SLVR/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [ ]:
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)


#lo que aqui se hace es guardar la informacion en un archivo json
#los datos se guardan en un arreglo, cada vez que se corre el codigo se agrega un nuevo objeto al arreglo

Data appended to data.json file.


In [ ]:
flight_arrivals = response.json()

for flight in flight_arrivals['arrivals']:
    scheduled_time = flight['arrival'].get('scheduledTime')
    number = flight.get('number')
    terminal = flight['arrival'].get('terminal')
    departure_airport = flight['departure']['airport'].get('name')
    aircraft_model = flight['aircraft'].get('model')
    airline_name = flight['airline'].get('name')
    # You can print or process these variables as needed

    #aqui modificamos lo que fue el codigo ya que mantenia errores y me ayudo a poder correr estas parte del cdigo

In [ ]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()


#aqui se crea un dataframe con la informacion obtenida de la API
#se utilizar para crear y exportar la informacion a un archivo CSV

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,OB 662,La Paz / El Alto,BoA,Boeing 737-800,2025-09-01 12:30Z,2025-09-01 08:30-04:00
0,OB 162,La Paz / El Alto,BoA,Bombardier CRJ,2025-09-01 12:50Z,2025-09-01 08:50-04:00
0,OB 638,Cochabamba,BoA,Boeing 737,2025-09-01 14:00Z,2025-09-01 10:00-04:00
0,OB 581,Sucre,BoA,Boeing 737-300,2025-09-01 14:45Z,2025-09-01 10:45-04:00
0,G3 7714,São Paulo,Gol,Boeing 737,2025-09-01 15:05Z,2025-09-01 11:05-04:00
